# Python notebook for post-processing ROI responses.
# Peak counting in the results data.
Assumes folder directory structure:
<pre><code>  IMAGING
    image_stacks
    notebooks
    results
</code></pre>
Execute the code sequentially, one block at a time, using &lt;shift-return&gt;.
#### Initialize.

In [ ]:
import csv
import glob
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.interpolate import interp1d
from scipy import signal
from scipy.optimize import curve_fit

# global variables
if os.name == "nt":
    FILE_SEP = "\\"
else:
    FILE_SEP = "/"


#### Select a results directory and peak counting options.


In [ ]:
%matplotlib widget

# global variables
results_sel = ""  # the selected results directory
stim_start = 100  # analysis start frame
stim_done = 250   # analysis end frame

pk_options = (0,0)
calc_latency = False
slope_type = 0
num_std = 0
baseline = (0,0)

s = {'description_width':'200px'} # a default widget style

# create results directory widget
result_dirs = sorted([f.split(FILE_SEP)[-2] for f in glob.glob("../results/*/", recursive=False)], key=str.casefold)
results_widget = widgets.Select(options=result_dirs, description='Results dir', 
                            disabled=False, layout=widgets.Layout(width='400px'))
# create numeric input widgets
stim_start_widget = widgets.BoundedIntText(value=stim_start, min=0, max=1000, step=1,
                    description='Stimulation start frame', disabled=False, layout={'width':'270px'}, style=s)
stim_done_widget = widgets.BoundedIntText(value=stim_done, min=0, max=1000, step=1,
                    description='Stimulation done frame', disabled=False, layout={'width':'270px'}, style=s)

# create status widget
status_widget = widgets.HTML(value=' ', description=' ')

# create sensitivity widget
sensitivity_widget = widgets.Dropdown(
    options=[('low',(0.06,4)),('low-medium',(0.05,6)),('medium',(0.04,7)),('high-medium',(0.03,8)),('high',(0.02,9))],
    value=(0.04,7),
    description='Peak counting sensitivity:',
    disabled=False,
    layout={'width':'350px'}, style={'description_width':'180px'}
)
#create latency widget
latency_widget = widgets.Checkbox(
    value=True,
    description='Calculate stimulation response latency details',
    disabled=False,
    indent=False
)
# create type of slope calculation widget
slope_widget = widgets.Dropdown(
    options = [('simple difference',(1)),('three-neighbour',(2)),('polynomial fit',(3)),('gaussian fit',(4))],
    value = 1,
    description='Slope calculation method:',
    disabled = False,
    layout={'width':'350px'}, style = {'description_width':'180px'}
)
# create number of standard deviations widget
std_widget = widgets.Dropdown(
    options = [('1',(1)),('2',(2)),('3',(3)),('4',(4)),('5',(5)),('6',(6)),('7',(7))],
    value = 2,
    description = 'Threshold standard deviations:',
    disabled = False,
    layout={'width':'260px'}, style = {'description_width':'200px'}
)
# display and respond to the widgets
def f(w1, w2, w3, w4, w5, w6, w7, w8):
  global results_sel
  global stim_start, stim_done
  global pk_options, calc_latency, slope_type, num_std, baseline
  results_sel = results_widget.value
  stim_start = stim_start_widget.value
  stim_done = stim_done_widget.value

  if not results_sel:
    status_widget.value = "No result directory selected."
  else:
    status_widget.value = "Selection OK."

  pk_options = sensitivity_widget.value
  calc_latency = latency_widget.value
  slope_type = slope_widget.value
  num_std = std_widget.value
display(widgets.interactive(f, w1=results_widget, w2=status_widget,
                           w3=stim_start_widget, w4=stim_done_widget,
                           w5=sensitivity_widget, w6=latency_widget, w7=slope_widget, w8 = std_widget))


#### Peak counting over all regions.
- The stimulation zone is marked with a white background color
- Peak points are marked with black color
- If latency is calculated:
 - latency response start points are marked with red color

In [ ]:
%matplotlib inline

def poly(x, a, b , c):
    return a * x + b * x**2 + c
def gauss(x, a, x0, sigma, y0):
    return y0 + (a * np.exp(-(x - x0)**2/(2*sigma**2)))

# get the regions and data labels in the selected results directory
results_dir = "../results/" + results_widget.value
region_files = os.listdir(results_dir)
region_files = sorted([f for f in region_files if f.startswith("region") and '.csv' in f])

# delete any previous results
for f in glob.glob(results_dir + "/peaks_*.csv"): os.remove(f)  
for f in glob.glob(results_dir + "/peaks_*.pdf"): os.remove(f)

with open("../results/" + results_widget.value + "/labels.txt") as f:
  data_labels = f.readlines()
data_labels = [x.strip() for x in data_labels] 

p = 2000         # desired length of resampled data
pkall = []       # the number of peaks across all stimulus zones regions
fpks = []        # first peak values           "               "
latencies = []   # latencies                   "               "
rise_times = []  # rise times                  "               "
max_slopes = []  # maximum slopes              "               "
zone_areas = []  # area under curve            "               "
max_vals = []    # maximum values              "               "

plt.close('all')
for r in region_files:     # for each region
  fig, ax = plt.subplots(nrows=1, ncols=1, figsize = [15,4])
  ax.grid(b=True)

  region = r.split('_')[-1].split('-')[0]  # get the region number
  A = np.transpose(np.genfromtxt("../results/" + results_sel + "/" + r, delimiter=','))

  tmin = np.min(A[0])      # start time
  tmax = np.max(A[0])      # finish time
  trng = tmax-tmin         # time range
  tstp = A[0,1]-A[0,0]     # time step
  dmin = np.min(A[1:])     # minimum data value (over all traces)
  dmax = np.max(A[1:])     # maximum data value
  drng = dmax-dmin         # data range
  X0 = A[0]                # the time axis
  Y0 = (A[1:]-dmin) / drng # data axis, normalized to range(0, 1.0)

  sr = p / trng   # the sample rate
  pk = []
  pts = []

  fpk = []      # first peak values     for this region
  lats = []     #    latencies       per stimulation zone, for this region
  riss = []     #    rise times          "                  "
  slps = []     #  maximum slopes        "                  "
  zars = []     # area under curve       "                  "
  maxs = []     # maximum values         "                  "
  for idx,y in enumerate(Y0): # for each trace
    f = interp1d(X0, y, kind='cubic')                     # define the resampling function
    X = np.linspace(tmin, tmax, p+1, endpoint=True)       # define the new time steps
    Y = f(X)                                              # resample the original signal

    # apply high-pass filter to eliminate the stimulation "bump" in the data
    sos = signal.butter(3, 0.1, btype='highpass', fs=sr, output='sos')
    Yf = signal.sosfiltfilt(sos, Y) # zero phase shift filter

    # apply low-pass filter to smooth out higher frequencies in the data
    sos = signal.butter(pk_options[1], 2.0, btype='lowpass', fs=sr, output='sos')
    Yf = signal.sosfiltfilt(sos, Yf) # zero phase shift filter

    # find all the peaks
    pks,_ = signal.find_peaks(Yf,prominence=pk_options[0])    # find indices of peaks in the resampled, filtered data
    pidx = np.around((A.shape[1]-1)*pks/p).astype(int)        # convert to indices in the original data
    pidx = [p for p in pidx if p>=stim_start and p<stim_done] # eliminate peaks outside the stimulation zone
    pk.append(len(pidx))                                       # save the number of peaks
    pts.append([A[0,pidx], A[idx+1,pidx]])                    # save the peaks as points in the original data

    ax.plot(A[0],A[idx+1],label=str(data_labels[idx]))    # plot the original data
    ax.plot(pts[-1][0],pts[-1][1],'k.')                   # plot the peak locations

    if calc_latency: # analyse "noise" in the region before stimulation (for latency calculation)
      signal_avg = np.mean(A[idx+1,0:stim_start])   # dc component of noisy signal
      noise = A[idx+1,0:stim_start] - signal_avg    # ac     "        "
      thresh = signal_avg + num_std * np.std(noise)  # latency threshold value
      mthresh = signal_avg - num_std * np.std(noise) # mirror threshold value around signal_avg 

      idxf = -1      # first peak index for this zone     
      idxl = -1      # latency start index for this zone 
      if len(pidx) > 0:                       # any peaks?
        idxf = pidx[0]                        # save the first peak index
        for i in range(stim_start, stim_done): # find latency start point
          if A[idx+1,i] >= thresh:            # above the threshold?
            ax.plot(A[0,i],A[idx+1,i],'r.')   # plot the latency start location
            idxl = i                          # save 
            break
      fpk.append(A[idx+1,idxf] if idxf!=-1 else 0)  # save first peak value

      lat = 0             # latency
      ris = 0             # rise time
      slp = 0             # max slope
      if (idxf != -1) and (idxl != -1):
        lat = A[0,idxl] - A[0,stim_start] # save latency time
        ris = A[0,idxf] - A[0,idxl]       # save rise time
        x_vals = A[0,idxl:idxf+1]
        y_vals = A[idx+1,idxl:idxf+1]
        if slope_type == 1:      # Simple difference calculations
          for i in range(len(x_vals)-1):
            sl = (y_vals[i+1] - y_vals[i]) / (x_vals[i+1] - x_vals[i])
            if sl > slp:
              slp = sl
        if slope_type == 2:      # Three neighbour numerical calculations
          if len(x_vals) > 5:
            for j in range(3, len(x_vals) - 3):
              sls = []
              for k in range(1,4):
                y1 = y_vals[j - k]
                y2 = y_vals[j + k]
                x1 = x_vals[j - k]
                x2 = x_vals[j + k]
                sls.append((y2 - y1) / (x2 - x1))
              sl = np.mean(sls)
              if sl > slp:
                slp = sl
        if slope_type == 3:       # Polynomial fit calculations
          if len(x_vals) > 3:
            popt, pcov = curve_fit(poly, x_vals, y_vals)
            fitted_y_vals = poly(x_vals, *popt)
            #ax.plot(x_vals,fitted_y_vals,'r')
            if len(fitted_y_vals) > 3:
              for i in range (1, len(fitted_y_vals)):
                sl = (fitted_y_vals[i] - fitted_y_vals[i - 1]) / (x_vals[i] - x_vals[i - 1])
                if sl > slp:
                  slp = sl
        if slope_type == 4:     # Gaussian fit calculations
          x_vals = A[0,idxl:idxf+1+len(x_vals)]     # extend selection of values to be symetrical aroung the peak
          y_vals = A[idx+1,idxl:idxf+1+len(y_vals)]   #
          if len(x_vals) > 6:
            mean = sum(x_vals * y_vals) / len(x_vals)
            sigma = sum(y_vals * (x_vals - mean)**2) / len(x_vals)
            y0 = thresh
            popt, pcov = curve_fit(gauss, x_vals, y_vals, p0=[max(y_vals),mean,sigma,y0],maxfev = 5000)
            fitted_y_vals = gauss(x_vals, *popt)
            #ax.plot(x_vals,fitted_y_vals,'r')
            peak_index = np.where(fitted_y_vals == np.max(fitted_y_vals))      # Find where peak of gaussian curve
            for i in range(1, int(peak_index[0])):                             # Find slopes up to peak of gaussian curve
              sl = (fitted_y_vals[i] - fitted_y_vals[i - 1]) / (x_vals[i] - x_vals[i - 1])
              if sl > slp:
                slp = sl
        
      lats.append(lat)
      riss.append(ris)
      slps.append(slp)        
    zars.append(np.trapz(A[idx+1,stim_start:stim_done] - signal_avg, A[0,stim_start:stim_done]))   # calculate and save area under curve
    maxs.append(np.amax(A[idx+1,stim_start:stim_done]))

  pkall.append(pk)         # save  the number of peaks across all regions
  fpks.append(fpk)         #  "      first peaks        "
  latencies.append(lats)   #  "      latencies          "
  rise_times.append(riss)  #  "     rise times          "
  max_slopes.append(slps)  #  "     maximum slopes      "
  zone_areas.append(zars)  #  "    area under curve     "
  max_vals.append(maxs)    #  "     maximum vlaues      "

  ax.legend()
  ax.set_title(" amplitude peaks - region " + region)
  
  xlab = "time (s)\n peak counts: " + str(pk)
  xlab += (", areas: ["+', '.join(['%.3g']*len(zars))+"]") % tuple(zars)
  xlab += (", max vals: ["+', '.join(['%.3g']*len(maxs))+"]") % tuple(maxs)
  if calc_latency:
    xlab += (",\nlatencies: ["+', '.join(['%.3g']*len(lats))+"]") % tuple(lats)
    xlab += (", rise times: ["+', '.join(['%.3g']*len(riss))+"]") % tuple(riss)
    xlab += (",\nfirst peak vals: ["+', '.join(['%.3g']*len(fpk))+"]") % tuple(fpk)
    xlab += (", max slopes: ["+', '.join(['%.3g']*len(slps))+"]") % tuple(slps)
  ax.set(xlabel=xlab)
  ax.set(ylabel="%F/F0")

  # grey out plot areas not in any zone
  ax.set_facecolor('0.8')
  ax.fill((A[0,stim_start],A[0,stim_done],A[0,stim_done],A[0,stim_start]),(dmin,dmin,dmax,dmax),'1.0')

  # save figure to pdf
  fig.savefig("../results/" + results_widget.value +'/' + "peaks" + "_region_" + region + ".pdf", bbox_inches='tight')
  plt.show()
  plt.close() # frees up memory

  # save peak point locations to CSV file
  with open ("../results/" + results_widget.value +'/' + "peaks" + "_region_" + region + ".csv", 'w') as file:
    writer = csv.writer(file)
    for trace in pts:                                      # for each trace...
      for pt in np.transpose(np.array(trace)):             #   for each point...
        writer.writerow('{:.3g}'.format(x) for x in pt)    #     save the point (time and intensity)
        
# save peak count and optional latency summary to CSV file
with open ("../results/" + results_widget.value + "/peaks_latencies_summary.csv", 'w') as file:
  writer = csv.writer(file)
  if calc_latency: 
    writer.writerow(("region number", "peak counts", "areas", "maximum values", "latencies", "rise times", "first peak values", "maximum slopes"))
  else: 
    writer.writerow(("region number", "peak counts", "areas", "maximum values"))
  for i in range(len(pkall)):
    r = ([i+1], pkall[i], zone_areas[i], max_vals[i], latencies[i], rise_times[i], fpks[i], max_slopes[i])
    writer.writerow('{:.3g}'.format(x) for x in tuple(np.concatenate(r))) 
     
    
